Połączenie z google drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Niebędne importy

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix, classification_report

Wczytanie danych

In [3]:
embeddings_path = '/content/drive/My Drive/data_sety/normalized_word2vec_embeddings.csv'
data_embeddings = pd.read_csv(embeddings_path)

original_data_path = '/content/drive/My Drive/data_sety/ready_data_set.csv'
original_data = pd.read_csv(original_data_path)
labels_true = original_data['label']


Podział danych na treningowe i testowe (będziemy trenować tylko i wyłącznie na wiadomościach typu nie spam)

In [4]:
X_train = data_embeddings[labels_true == 0]
y_train = labels_true[labels_true == 0]
X_test = data_embeddings
y_test = labels_true

Wykrywanie i inicjalizacja TPU

In [5]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.TPUStrategy(resolver)

Ustawienia modelu

In [6]:
# Liczba cech
input_dim = X_train.shape[1]

# Liczba neuronów w warstwie kodowania
encoding_dim = 32

Funckja budująca autoencoder

In [7]:
def build_autoencoder(input_dim, encoding_dim):
    # Ustawienie warstwy wejściowej
    input_layer = Input(shape=(input_dim,))

    # Ustawienie warstwy enkodowania
    encoded = Dense(encoding_dim, activation='relu')(input_layer)

    # Ustawienie warstwy dekodowania
    decoded = Dense(input_dim, activation='sigmoid')(encoded)

    # Budowanie autoencodera
    autoencoder = Model(input_layer, decoded)

    # Kompilacja modelu autoencodera z optymalizatorem adam oraz funkcją straty mse
    autoencoder.compile(optimizer='adam', loss='mse')

    # Zwrócenie autoencodera
    return autoencoder

5-krotna walidacja krzyżowa. Sprawdamy tutaj jak dobrze autoencoder uczy się rekonstrukcji wiadomości typu nie spam na zbiorze treningowym.

Do oceny używamy średniego błędu kwadratowego - MSE oraz wbudowanego w sklearn raportu klasyfikacji (wyświetla on metryki związane z klasyfikacją) oraz macierzy konfuzji (pokazuje ona ile przewidywań każdego rodzaju (True Positives, False Positives, True Negatives, False Negatives) zostało wykonanych dla każdej klasy).

In [8]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
mse_train = []
mse_val = []
mse_test = []
all_y_pred = []
all_y_true = []

In [9]:
for fold, (train_index, val_index) in enumerate(kf.split(X_train)):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]

    with strategy.scope():
        # Budowanie autoencodera
        autoencoder = build_autoencoder(input_dim, encoding_dim)


        history = autoencoder.fit(X_train_fold, X_train_fold, # Trenujemy autoencoder na danych treningowych.
                                  epochs=50, # Liczba pełnych przebiegów przez dane treningowe.
                                  batch_size=256, # Liczba próbek przetwarzanych przed zaktualizowaniem modelu.
                                  shuffle=True, # Przypadkowe mieszanie danych przed każdą epoką.
                                  validation_data=(X_val_fold, X_val_fold), # Dane walidacyjne używane do oceny modelu w trakcie treningu.
                                  verbose=0) # Ukrywanie szczegółowych informacji o przzebiegu treningu.

    # Dodanie do listy błędu kwadratowego (MSE) dla aktualnej iteracji
    mse_train.append(history.history['loss'])
    mse_val.append(history.history['val_loss'])

    # Rekonstrukcja danych walidacyjnych na danych testowych (czyli pełny zbiór danych)
    reconstructions = autoencoder.predict(X_test) # Zrekonstrułowane dane testowe (każdy wiersza zawiera wiadomości a w każdej wiadomości są zrekonstrułowane pokolei słowa)
    reconstruction_errors = np.mean(np.square(reconstructions - X_test), axis=1) # Obliczenie dla każdego zdania średniego błędu kwadratowego dla każdego/wiersza wiadomości
    mse_test.append(np.mean(reconstruction_errors)) # Oblicza średni błąd kwadratowy

    # Ustal próg rekonstrukcji na podstawie błędów na danych treningowych
    train_reconstructions = autoencoder.predict(X_train_fold)
    train_reconstruction_errors = np.mean(np.square(train_reconstructions - X_train_fold), axis=1)
    threshold = np.percentile(train_reconstruction_errors, 95) # Oblicza percentyl z train_reconstruction_errors (wyznacza wartość dokładności rekonstrukcji powyżej której znajduje się 95% wszystki wiadomości)

    # Klasyfikacja wiadomości jako spam lub nie-spam na podstawie błędów rekonstrukcji
    y_pred_fold = reconstruction_errors > threshold
    all_y_pred.extend(y_pred_fold) # Przewidziane etykiety
    all_y_true.extend(y_test) # Prawdziwe etykiety

    # Wyświetlenie średniego MSE dla bieżącego folda
    print("Fold" + str(fold + 1))
    print("Średni MSE dla treningu: " + str(np.mean(history.history['loss'])))
    print("Średni MSE dla walidacji: " + str(np.mean(history.history['val_loss'])))
    print("Średni MSE dla danych testowych: " + str(np.mean(reconstruction_errors)))

    # Wyświetlenie raportu klasyfikacji i macierzy konfuzji dla bieżącego folda
    print("Raport klasyfikacji dla bieżącego folda:")
    print(classification_report(y_test, y_pred_fold, target_names=['Not Spam', 'Spam']))

    print("Macierz konfuzji dla bieżącego folda:")
    print(confusion_matrix(y_test, y_pred_fold))

2554/2554 [==============================] - 21s 8ms/step
Fold1
Średni MSE dla treningu: 0.006600839067250491
Średni MSE dla walidacji: 0.005673279268667102
Średni MSE dla danych testowych: 0.005428586393756369
Raport klasyfikacji dla bieżącego folda:
              precision    recall  f1-score   support

    Not Spam       0.57      0.95      0.71    102159
        Spam       0.78      0.20      0.32     91689

    accuracy                           0.60    193848
   macro avg       0.68      0.57      0.51    193848
weighted avg       0.67      0.60      0.53    193848

Macierz konfuzji dla bieżącego folda:
[[97074  5085]
 [73402 18287]]
2554/2554 [==============================] - 20s 7ms/step
Fold2
Średni MSE dla treningu: 0.006582772377878427
Średni MSE dla walidacji: 0.0056814838480204344
Średni MSE dla danych testowych: 0.005438010403606508
Raport klasyfikacji dla bieżącego folda:
              precision    recall  f1-score   support

    Not Spam       0.57      0.95      0.71 

Przekształcenie do tablicy numpy dla przewidzianych i prawdziwych etykiet

In [10]:
all_y_pred_nump = np.array(all_y_pred)
all_y_true_nump = np.array(all_y_true)

Wygenerowanie raportu

In [11]:
classification_report_str = classification_report(all_y_true_nump, all_y_pred_nump, target_names=['Not Spam', 'Spam'])
confusion_matrix_str = str(confusion_matrix(all_y_true_nump, all_y_pred_nump))

In [12]:
mean_mse_train = np.mean([np.mean(mse) for mse in mse_train])
mean_mse_val = np.mean([np.mean(mse) for mse in mse_val])
mean_mse_test = np.mean(mse_test)

Wyświetlenie podsumowania

In [13]:
# Obliczanie średniego MSE ze wszystkicj foldów
print("Średni MSE dla treningu dla wszystkich foldów: " + str(mean_mse_train))
print("Średni MSE dla walidacji dla wszystkich foldów: " + str(mean_mse_val))
print("Średni MSE dla danych testowych dla wszystkich foldów: " + str(mean_mse_test))

# Raport klasyfikacji na danych walidacyjnych ze wszystkich foldów.
print(classification_report_str)

# Macierz pomyłek na danych walidacyjnych ze wszystkich foldów
print(confusion_matrix_str)

Średni MSE dla treningu dla wszystkich foldów: 0.00657881859317422
Średni MSE dla walidacji dla wszystkich foldów: 0.005686780704185367
Średni MSE dla danych testowych dla wszystkich foldów: 0.005445704891314047
              precision    recall  f1-score   support

    Not Spam       0.57      0.95      0.71    510795
        Spam       0.78      0.20      0.32    458445

    accuracy                           0.60    969240
   macro avg       0.68      0.58      0.52    969240
weighted avg       0.67      0.60      0.53    969240

[[485246  25549]
 [365454  92991]]


Zapisanie wyników do pliku

In [14]:
with open('/content/drive/My Drive/data_sety/folds_keras_word2vec_validation_classification_report.txt', 'w') as f:
    f.write("Raport klasyfikacji na danych walidacyjnych:\n")
    f.write(classification_report_str)
    f.write("\nMacierz konfuzji na danych walidacyjnych:\n")
    f.write(confusion_matrix_str)

In [15]:
with open('/content/drive/My Drive/data_sety/folds_keras_word2vec_validation_mse.txt', 'w') as f:
    f.write("Średni MSE dla treningu dla wszystkich foldów: " + str(mean_mse_train) + "\n")
    f.write("Średni MSE dla walidacji dla wszystkich foldów: " + str(mean_mse_val) + "\n")
    f.write("Średni MSE dla danych testowych dla wszystkich foldów: " + str(mean_mse_test) + "\n")

Trenowanie autoencodera na całym zbiorze treningowym

In [16]:
with strategy.scope():
    autoencoder = build_autoencoder(input_dim, encoding_dim)
    history = autoencoder.fit(X_train, X_train,
                              epochs=50,
                              batch_size=256,
                              shuffle=True,
                              validation_split=0.2,
                              verbose=1)

Epoch 1/50
320/320 [==============================] - 7s 16ms/step - loss: 0.0511 - val_loss: 0.0081
Epoch 2/50
320/320 [==============================] - 4s 12ms/step - loss: 0.0078 - val_loss: 0.0071
Epoch 3/50
320/320 [==============================] - 4s 12ms/step - loss: 0.0071 - val_loss: 0.0067
Epoch 4/50
320/320 [==============================] - 4s 12ms/step - loss: 0.0068 - val_loss: 0.0065
Epoch 5/50
320/320 [==============================] - 4s 12ms/step - loss: 0.0065 - val_loss: 0.0063
Epoch 6/50
320/320 [==============================] - 4s 13ms/step - loss: 0.0064 - val_loss: 0.0062
Epoch 7/50
320/320 [==============================] - 4s 12ms/step - loss: 0.0063 - val_loss: 0.0061
Epoch 8/50
320/320 [==============================] - 4s 12ms/step - loss: 0.0062 - val_loss: 0.0060
Epoch 9/50
320/320 [==============================] - 4s 12ms/step - loss: 0.0061 - val_loss: 0.0059
Epoch 10/50
320/320 [==============================] - 4s 12ms/step - loss: 0.0060 - val_lo

Zapisanie autoencodera do pliku

In [17]:
autoencoder.save('/content/drive/My Drive/data_sety/final_keras_word2vec_autoencoder_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Rekonstrukcja danych testowych

In [18]:
reconstructions = autoencoder.predict(X_test)
reconstruction_errors = np.mean(np.square(reconstructions - X_test), axis=1)
mse_test = []
mse_test.append(np.mean(reconstruction_errors))

6058/6058 [==============================] - 48s 8ms/step


Ustal próg rekonstrukcji na podstawie błędów na danych treningowych

In [19]:
train_reconstructions = autoencoder.predict(X_train)
train_reconstruction_errors = np.mean(np.square(train_reconstructions - X_train), axis=1)
threshold = np.percentile(train_reconstruction_errors, 95)

3193/3193 [==============================] - 25s 8ms/step


Klasyfikacja jako spam lub nie-spam na podstawie błędów rekonstrukcji

In [20]:
y_pred = reconstruction_errors > threshold

Raport klasyfikacji i macierzz pomyłek oraz MSE dla treningu na całym zbiorze

In [21]:
classification_report_test_str = classification_report(y_test, y_pred, target_names=['Not Spam', 'Spam'])
confusion_matrix_test_str = str(confusion_matrix(y_test, y_pred))

In [22]:
mean_mse_train = np.mean(history.history['loss'])
mean_mse_val = np.mean(history.history['val_loss'])
mean_mse_test = np.mean(mse_test)

In [23]:
print("średni MSE dla treningu: " + str(mean_mse_train))
print("Średni MSE dla walidacji: " + str(mean_mse_val))
print("Średni MSE dla danych testowych: " + str(mean_mse_test))

print(classification_report_test_str)
print(confusion_matrix_test_str)

średni MSE dla treningu: 0.006590225920081139
Średni MSE dla walidacji: 0.005599378058686853
Średni MSE dla danych testowych: 0.005443098342799986
              precision    recall  f1-score   support

    Not Spam       0.57      0.95      0.71    102159
        Spam       0.78      0.20      0.31     91689

    accuracy                           0.59    193848
   macro avg       0.67      0.57      0.51    193848
weighted avg       0.67      0.59      0.52    193848

[[97051  5108]
 [73765 17924]]


Zapisanie danych do pliku

In [24]:
with open('/content/drive/My Drive/data_sety/final_keras_word2vec_validation_classification_report.txt', 'w') as f:
    f.write("Raport klasyfikacji na danych testowych:\n")
    f.write(classification_report_test_str)
    f.write("\nMacierz konfuzji na danych testowych:\n")
    f.write(confusion_matrix_test_str)

In [25]:
with open('/content/drive/My Drive/data_sety/final_keras_word2vec_validation_mse.txt', 'w') as f:
    f.write("Średni MSE dla treningu: " + str(mean_mse_train) + "\n")
    f.write("Średni MSE dla walidacji: " + str(mean_mse_val) + "\n")
    f.write("Średni MSE dla danych testowych: " + str(mean_mse_test) + "\n")